# オルソマッピング ガイド (Part 3)

<h1>目次<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Part-2-のおさらい" data-toc-modified-id="Continued-from-Part-2-1">Part 2 のおさらい</a></span></li><li><span><a href="#Step-4.-画像の管理" data-toc-modified-id="Step-4.-Managing-imagery-2">Step 4. 画像の管理</a></span><ul class="toc-item"><li><span><a href="#4.1-画像コレクションの処理状況の取得・設定" data-toc-modified-id="4.1-Get-or-set-image-collection-processing-status-2.1">4.1 画像コレクションの処理状況の取得・設定</a></span></li><li><span><a href="#4.2-画像コレクションに画像を追加" data-toc-modified-id="4.2-Add-images-to-image-collection-2.2">4.2 画像コレクションに画像を追加</a></span></li><li><span><a href="#4.3-画像コレクションから画像を削除" data-toc-modified-id="4.3-Delete-images-from-image-collection-2.3">4.3 画像コレクションから画像を削除</a></span></li><li><span><a href="#4.4-画像コレクションのリセット" data-toc-modified-id="4.4-Reset-image-collection-2.4">4.4 画像コレクションのリセット</a></span></li><li><span><a href="#4.5-画像コレクションの削除" data-toc-modified-id="4.5-Delete-image-collection-2.5">4.5 画像コレクションの削除</a></span></li></ul></li><li><span><a href="#まとめ" data-toc-modified-id="Conclusion-3">まとめ</a></span></li><li><span><a href="#参考" data-toc-modified-id="References-4">参考</a></span></li></ul></div>

## Part 2 のおさらい
これまで、Part1 では、オルソマッピングの概念、オルソマッピングツールが動作するための環境設定、画像の整理方法、画像コレクション レイヤー アイテムの作成方法を紹介し、Part2 では、ブロック調整、コントロール ポイントの操作、シームラインやカラー コレクションの計算方法、オルソモザイク、DEM、植生指標の生成手順について紹介してきました。

Part 3 では、処理した画像の扱い方について、画像コレクションへの画像の追加／削除、画像コレクションのリセット／削除の方法について説明します。

まず、GISインスタンスに接続します。

In [1]:
import arcgis
from arcgis.gis import GIS

portal_url = "https://yourportal.domain.com/portal"
portal_username = "your username"
portal_password = "your password"

gis = GIS(url=portal_url, username=portal_username, password=portal_password)

画像コレクション レイヤー アイテムは、Part1 で既に作成されていますので、ここでは、レイヤー アイテムの参照を再利用しましょう。

In [3]:
image_collection_item = gis.content.search("omImageCollection")[0]
image_collection_item.url

'https://yourportal.domain.com/server/rest/services/Hosted/omImageCollection/ImageServer'

## Step 4. 画像の管理

### 4.1 画像コレクションの処理状況の取得・設定

[arcgis.raster.orthomapping.get_processing_states()](https://developers.arcgis.com/python/api-reference/arcgis.raster.orthomapping.html#get-processing-states) および [arcgis.raster.orthomapping.alter_processing_states()](https://developers.arcgis.com/python/api-reference/arcgis.raster.orthomapping.html#alter-processing-states) は、処理状態の照会と変更に使用します。処理状態は、通常画像収集時に実行されたオルソマッピング処理に基づいて設定されます。これは、その後のオルソマッピング ワークフローでデータがどのように扱われるかを決定するため重要なことです。

In [45]:
from arcgis.raster.orthomapping import get_processing_states
get_processing_states(image_collection=image_collection_item)

{'blockadjustment': 'full',
 'dem': {'match_mode': 'ETM',
  'object_size': 15,
  'ground_spacing': None,
  'minimum_pairs': 8,
  'minimum_adjustment_quality': 0.2,
  'maximum_diff_gsd': 2,
  'maximum_diff_OP': 8,
  'interpolation_method': 'TRIANGULATION',
  'smooth_method': 'GAUSS5X5',
  'surface_type': 'DSM'},
 'seamlines': {'computation_method': 'DISPARITY',
  'blend_width': None,
  'blend_type': 'BOTH',
  'request_size': 1000,
  'request_size_type': 'PIXELS',
  'blend_width_units': 'PIXELS',
  'min_region_size': 100,
  'min_thinness_ratio': 0.05,
  'max_sliver_size': 20},
 'colorcorrection': {'balancing_method': 'DODGING',
  'surface_type': 'SINGLE_COLOR'},
 'adjust_index': 2,
 'imagetype': 'UAV/UAS'}

In [46]:
from arcgis.raster.orthomapping import alter_processing_states

new_states ={'blockadjustment': 'quick',
             'dem': '',
             'seamlines':{'computation_method': 'DISPARITY',
                          'blend_width': None,
                          'blend_type': 'BOTH',
                          'request_size': 1000,
                          'request_size_type': 'PIXELS',
                          'blend_width_units': 'PIXELS',
                          'min_region_size': 100,
                          'min_thinness_ratio': 0.05,
                          'max_sliver_size': 20},
             'colorcorrection': {'balancing_method': 'DODGING',
                                 'surface_type': 'SINGLE_COLOR'},
             'adjust_index': 1,
             'imagetype': 'UAV/UAS'}

alter_processing_states(image_collection=image_collection_item, new_states=new_states)

{'orthomapping': {'blockadjustment': 'quick',
  'dem': '',
  'seamlines': {'computation_method': 'DISPARITY',
   'blend_width': None,
   'blend_type': 'BOTH',
   'request_size': 1000,
   'request_size_type': 'PIXELS',
   'blend_width_units': 'PIXELS',
   'min_region_size': 100,
   'min_thinness_ratio': 0.05,
   'max_sliver_size': 20},
  'colorcorrection': {'balancing_method': 'DODGING',
   'surface_type': 'SINGLE_COLOR'},
  'adjust_index': 1,
  'imagetype': 'UAV/UAS'}}

### 4.2 画像コレクションに画像を追加

[arcgis.raster.analytics.add_image()](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#add-image) は、既存の画像コレクションに新しい画像を追加するのに役立ちます。これは、既存のシームライン、フットプリント、コントロール ポイントなど、他の画像に関連するものを変更するものではありません。

In [47]:
from arcgis.raster.analytics import add_image

In [48]:
image_item = gis.content.search("YUN_0040.JPG")[0]

In [49]:
raster_type_params = {
    "gps":[["YUN_0040.JPG",34.006989,-117.09279,725.13]], "isAltitudeFlightHeight":"False",
    "cameraProperties":{"maker":"Yuneec", "model":"E90", "focallength":8,"columns":5472, "rows":3648, "pixelsize":0.0024},
    "averagezdem":{
        "url":"https://elevation3d.arcgis.com/arcgis/rest/services/WorldElevation3D/Terrain3D/ImageServer"
    }
}

In [1]:
add_image(image_collection=image_collection_item, input_rasters=[image_item], raster_type_name="UAV/UAS", 
          raster_type_params=raster_type_params)

'https://yourportal.domain.com/server/rest/services/Hosted/omImageCollection/ImageServer'

### 4.3 画像コレクションから画像を削除

[arcgis.raster.analytics.delete_image()](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#delete-image) は、画像コレクションから既存の画像を削除するのに役立ちます。これは、既存のシームライン、フットプリント、コントロール ポイントなど、他の画像に関連するものを変更するものではありません。

In [51]:
from arcgis.raster.analytics import delete_image

In [2]:
delete_image(image_collection=image_collection_item, where="objectID=5")

'https://yourportal.domain.com/server/rest/services/Hosted/omImageCollection/ImageServer'

### 4.4 画像コレクションのリセット

[arcgis.raster.orthomapping.reset_image_collection()](https://developers.arcgis.com/python/api-reference/arcgis.raster.orthomapping.html#reset-image-collection) は、画像コレクションのイメージ レイヤー アイテムを元の状態にリセットするのに役立ちます。

これは、画像に対してすでに計算されたブロック調整結果をリセットし、フットプリントを再構築し、シームラインをリセットし、色補正結果を削除します。

以前のブロック調整で計算された既存のコントロール ポイント フィーチャクラス、飛行経路フィーチャクラス等は削除されません。

また、調整済み画像コレクションから派生したプロダクトを削除することはありません。

In [53]:
from arcgis.raster.orthomapping import reset_image_collection

In [54]:
reset_image_collection(image_collection=image_collection_item)

True

### 4.5 画像コレクションの削除

[arcgis.raster.analytics.delete_image_collection()](https://developers.arcgis.com/python/api-reference/arcgis.raster.analytics.html#delete-image-collection) は、ArcGIS Enterprise から画像コレクションのイメージ レイヤー アイテムを削除するのに役立ちます。

画像コレクションと一緒に保存されていない、その他の関連する結果や製品はすべて残ります。

In [55]:
from arcgis.raster.analytics import delete_image_collection

In [56]:
delete_image_collection(image_collection=image_collection_item)

True

## まとめ

このガイドでは、画像の処理とオルソマッピングの生成において、4 つのステップを実行しました。

 - __Step 1. 準備__ では、複数の画像から画像コレクション レイヤー アイテムを作成する方法を紹介しました。
 - __Step 2. 調整__ では、ブロック調整、コントロール ポイントの操作、シームラインやカラー コレクションの計算方法を紹介しました。
 - __Step 3. 結果の取得__ では、オルソモザイク、DEM、植生指標をオルソマッピングツールで作成する手順について紹介しました。
 - __Step 4. 画像の管理__ では、画像コレクションの追加/削除、画像コレクションのリセット/削除を行う方法を紹介しました。

DTM や DSM などのオルソマッピング製品や、航空デジタル画像などを使ったオルソモザイクを Python API で取得することで、標高データを使った様々な解析や、オルソモザイク地図を使った緊急対応、資源管理、不動産などのアプリケーションに利用することができます<a href='#参考'>[1]</a>。

## 参考

<a href="#Conclusion">[1]</a> Lenhardt, Ortho Mapping in ArcGIS Pro Part III: Getting Results,https://www.esri.com/arcgis-blog/products/arcgis-pro/imagery/ortho-mapping-products-arcgis-pro/ [Online]